# Assignment 2

As before, if a question can be answered with 'yes/no', or a numeric value, you may simply state as much. If you incorporate code from the internet (which is not required and generally not advisable), please cite the source within your code (providing a URL is sufficient).

We will go through comparable code and concepts in the live learning sessions. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that no outside searches are required by the assignment!). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Import specific objects
from sklearn.preprocessing import StandardScaler
from ISLP import load_data

### Question 1: Classification using KNN

We'll now use the `Caravan` dataset from the `ISLP` package. (You may use `Caravan.describe()` to review details of the dataset.) In this dataset, the response variable of interest is `Purchase`, which indicates if a given customer purchased a caravan insurance policy. We will simultaneously use all other variables in the dataset to predict the response variable.

In [17]:
# Load the "Caravan" dataset using the "load_data" function from the ISLP package
Caravan = load_data('Caravan')
Caravan

,MOSTYPE,MAANTHUI,MGEMOMV,MGEMLEEF,MOSHOOFD,MGODRK,MGODPR,MGODOV,MGODGE,MRELGE,...,APERSONG,AGEZONG,AWAOREG,ABRAND,AZEILPL,APLEZIER,AFIETS,AINBOED,ABYSTAND,Purchase
0,33,1,3,2,8,0,5,1,3,7,...,0,0,0,1,0,0,0,0,0,No
1,37,1,2,2,8,1,4,1,4,6,...,0,0,0,1,0,0,0,0,0,No
2,37,1,2,2,8,0,4,2,4,3,...,0,0,0,1,0,0,0,0,0,No
3,9,1,3,3,3,2,3,2,4,5,...,0,0,0,1,0,0,0,0,0,No
4,40,1,4,2,10,1,4,1,4,7,...,0,0,0,1,0,0,0,0,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5817,36,1,1,2,8,0,6,1,2,1,...,0,0,0,1,0,0,0,0,0,No
5818,35,1,4,4,8,1,4,1,4,6,...,0,0,0,1,0,0,0,0,0,No
5819,33,1,3,4,8,0,6,0,3,5,...,0,0,0,1,0,0,0,0,0,Yes
5820,34,1,3,2,8,0,7,0,2,7,...,0,0,0,0,0,0,0,0,0,No


Before fitting any model, it is essential to understand our data. Answer the following questions about the `Caravan` dataset (Hint: use `print` and `describe`):  
_(i)_ How many observations (rows) does the dataset contain? 

5821 

_(ii)_ How many variables (columns) does the dataset contain?

86 including the predictor variable. 

_(iii)_ What 'variable' type is the response variable `Purchase` (e.g., 'character', 'factor', 'numeric', etc)? What are the 'levels' of the variable? 

variable type is character. 
levels of the variables are "Yes" and "No"

_(iv)_ How many predictor variables do we have (Hint: all variables other than `Purchase`)?  

85

In [18]:
Purchase = Caravan.Purchase
Purchase.value_counts()

Purchase
No     5474
Yes     348
Name: count, dtype: int64

Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the `scaler` method, provided as follows:

In [19]:
# Select predictors (excluding the 86th column)
predictors = Caravan.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head())

    MOSTYPE  MAANTHUI   MGEMOMV  MGEMLEEF  MOSHOOFD    MGODRK    MGODPR  \
0  0.680906  -0.27258  0.406697 -1.216964  0.779405 -0.694311  0.217444   
1  0.992297  -0.27258 -0.859500 -1.216964  0.779405  0.302552 -0.365410   
2  0.992297  -0.27258 -0.859500 -1.216964  0.779405 -0.694311 -0.365410   
3 -1.187437  -0.27258  0.406697  0.010755 -0.970980  1.299414 -0.948264   
4  1.225840  -0.27258  1.672893 -1.216964  1.479559  0.302552 -0.365410   

     MGODOV    MGODGE    MRELGE  ...   ALEVEN  APERSONG   AGEZONG  AWAOREG  \
0 -0.068711 -0.161816  0.427670  ... -0.20291 -0.073165 -0.081055 -0.05992   
1 -0.068711  0.464159 -0.096077  ... -0.20291 -0.073165 -0.081055 -0.05992   
2  0.914172  0.464159 -1.667319  ... -0.20291 -0.073165 -0.081055 -0.05992   
3  0.914172  0.464159 -0.619824  ... -0.20291 -0.073165 -0.081055 -0.05992   
4 -0.068711  0.464159  0.427670  ... -0.20291 -0.073165 -0.081055 -0.05992   

     ABRAND   AZEILPL  APLEZIER   AFIETS   AINBOED  ABYSTAND  
0  0.764971 -0.02

_(v)_ Why is it important to standardize the predictor variables?  

Standardization ensures that all predictor variables are on the same scale.

_(vi)_ Why did we elect not to standard our response variable `Purchase`?  

Standardizing the response variable would change its scale, making it difficult to interpret the model coefficients in terms of the original units of the response variable. 

In [20]:
# Your answer here


_(vii)_ A second essential step is to set a random seed. Do so below (Hint: use the `random.seed` function). Why is setting a seed important? Is the particular seed value important? Why or why not?

The seed value itself is not important. It can be any value. However, the important thing to consider is that the seed value should remain constant every time one is doing the same analysis. 

In [21]:
np.random.seed(42)

_(viii)_ A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. Extend the code to create a non-overlapping test set for the predictors and response variables.

In [22]:
# Create a random vector of True and False values
split = np.random.choice([True, False], size=len(predictors_standardized), replace=True, p=[0.75, 0.25])

# Define the training set for X (predictors)
training_X = predictors_standardized[split]

# Define the training set for Y (response)
training_Y = Caravan.loc[split, 'Purchase']

# Define the testing set for X (predictors)
testing_X = predictors_standardized[~split]

# Define the testing set for Y (response)
testing_Y = Caravan.loc[~split, 'Purchase']


_(ix)_ We are finally set to fit the KNN model. In Python, we can use the `KNeighborsClassifier()` function. Fit the KNN with k=1. (You may review arguments to knn by typing `help(knn.fit)`). 

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import pandas as pd

# Assuming training_X, training_Y, testing_X, and testing_Y are your training and testing data
# and labels respectively
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(training_X, training_Y)

knn1_pred = knn1.predict(testing_X)

# Generate confusion matrix
confusion_table = confusion_matrix(testing_Y, knn1_pred)

# Convert the array into a pandas DataFrame with labels for better readability
confusion_table_df = pd.DataFrame(confusion_table, 
                                  columns=['Predicted No', 'Predicted Yes'], 
                                  index=['Actual No', 'Actual Yes'])

print("Confusion Matrix:")
print(confusion_table_df)


Confusion Matrix:
            Predicted No  Predicted Yes
Actual No           1280             80
Actual Yes            70              9


Using your fit model, answer the following questions:   
_(x)_ What is the prediction accuracy? (Hint: use the `score` method, and compare your model to `testing_Y`)  

Accuracy of the KNN model: 0.8957609451007644

_(xi)_ What is the predictor error ? (Hint: compute it from the accuracy)

Prediction Error of the KNN model: 0.10423905489923557


In [24]:
# prediction accuracy rate
from sklearn.metrics import accuracy_score

# Calculate the accuracy of the KNN model
accuracy = accuracy_score(testing_Y, knn1_pred)

print("Accuracy of the KNN model:", accuracy)



Accuracy of the KNN model: 0.8957609451007644


In [25]:
# prediction error rate
prediction_error = 1 - accuracy

print("Prediction Error of the KNN model:", prediction_error)

Prediction Error of the KNN model: 0.10423905489923557


_(xii)_ How does this prediction error/accuracy compare to what could be achieved via random guesses? To answer this, consider the percent of customers in the `Caravan` dataset who actually purchase insurance, computed below:

The calculation for guess for people who purchased insurance is about 6%. On the other hand the prediction 
6.18%. Which is very close

In [26]:
# Calculate the percentage of customers who purchase insurance
percentage_purchase = Caravan['Purchase'].eq('Yes').sum() / (Caravan['Purchase'].eq('No').sum() + Caravan['Purchase'].eq('Yes').sum()) * 100

print(percentage_purchase)

5.977327378907591


_(xiii)_ Fit a second KNN model, with $K=3$. Does this model perform better (i.e., have higher accuracy, compared to a random guess)? 

With K=3 the predictive positive is 2.18% which is not close to 6% of random guess.

This definitely improved the resuts and came close to the results between random guess vs. actual results

In [27]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
import pandas as pd

# Assuming training_X, training_Y, testing_X, and testing_Y are your training and testing data
# and labels respectively
knn1 = KNeighborsClassifier(n_neighbors=3)
knn1.fit(training_X, training_Y)

knn1_pred = knn1.predict(testing_X)

# Generate confusion matrix
confusion_table = confusion_matrix(testing_Y, knn1_pred)

# Convert the array into a pandas DataFrame with labels for better readability
confusion_table_df = pd.DataFrame(confusion_table, 
                                  columns=['Predicted No', 'Predicted Yes'], 
                                  index=['Actual No', 'Actual Yes'])

print("Confusion Matrix:")
print(confusion_table_df)

Confusion Matrix:
            Predicted No  Predicted Yes
Actual No           1334             26
Actual Yes            76              3


In [28]:
# prediction accuracy rate
from sklearn.metrics import accuracy_score

# Calculate the accuracy of the KNN model
accuracy = accuracy_score(testing_Y, knn1_pred)

print("Accuracy of the KNN model:", accuracy)


Accuracy of the KNN model: 0.9291174426685198


In [29]:
# prediction error rate
prediction_error = 1 - accuracy

print("Prediction Error of the KNN model:", prediction_error)

Prediction Error of the KNN model: 0.07088255733148019


# Criteria

|Criteria            |Complete           |Incomplete          |
|--------------------|---------------|--------------|
|Classification using KNN|All steps are done correctly and the answers are correct.|At least one step is done incorrectly leading to a wrong answer.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/applied_statistical_concepts/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
